In [3]:
from bs4 import BeautifulSoup as bs
import requests
from urllib.parse import quote
import pandas as pd
import re

headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.93 Safari/537.36'}



In [13]:
def get_news(query, page_num=10):
    
    news_df = pd.DataFrame(columns=("Title", "Link", "Press", "DataeTime", "Article"))
    idx = 0
    
    url_query = quote(query)
    url = "https://search.naver.com/search.naver?where=news&sm=tab_jum&query=" + url_query
    
    for _ in range(0, page_num):
        
        search_url = requests.get(url, headers=headers)
        soup = bs(search_url.content, 'html.parser')
        links = soup.findAll('div', class_="info_group")
        
        for link in links:
            # news_url = link.select_one('.info_group a[href]:last-child').attrs['href']

            firststep = link.find_all("a")
            press = firststep[0].text  #언론사
            if "언론사 선정" in press:
                press = press.replace("언론사 선정", "")

            naver_news_only_arr_news_url = link.select_one('.info_group a.info:last-child')
            if (naver_news_only_arr_news_url == None):
                continue
            else:
    #             print(naver_news_only_arr_news_url.get('href')) # 네이버 뉴스 링크 
                news_url = naver_news_only_arr_news_url.get('href')
    #             print(news_url)
                news_link = requests.get(news_url, headers=headers)
                news_html = bs(news_link.content, 'html.parser')
            
                title = news_html.find('h3', {'id':'articleTitle'})
                if (title != None):
                    title = title.get_text()
                
                datetitme = news_html.find('span', {'class':'t11'})
                if (datetitme != None):
                    datetitme = datetitme.get_text()
                    
                article = news_html.find('div', {'id':'articleBodyContents'})
                if (article != None):
                    article = article.get_text()

                news_df.loc[idx] = [title, news_url, press, datetitme, article]
                idx += 1
                print("#", end ="")
    
        try:
            next_page = soup.find('a', {'class':'btn_next'}).get('href')
            url = "https://search.naver.com/search.naver" + next_page
        except:
            break
        
    return news_df
            

In [15]:
query = input("검색어: ")
news_df = get_news(query, 1)
print("검색 완료!")
# get_news("인공지능")

검색어: 인공지능
#######검색 완료!


In [16]:
news_df

,Title,Link,Press,DataeTime,Article
0,광주 서구 '인공지능 통합돌봄 사업' 좋은정책 대상,https://news.naver.com/main/read.naver?mode=LS...,연합뉴스,2021.12.28. 오후 3:46,\n\n\n\n\n광주 서구청 전경[광주 서구청 제공] (광주=연합뉴스) 천정...
1,"과기정통부, 내년 기술패권시대 대응에 '올인'…반도체·인공지능 등 집중육성",https://news.naver.com/main/read.naver?mode=LS...,경향신문,2021.12.28. 오전 11:27,\n\n\n\n\n\t\n\t[경향신문] 내년 과학기술정보통신부 업무계획 요약. 과...
2,터널 속 미세 균열...인공지능 로봇으로 탐지!,https://news.naver.com/main/read.naver?mode=LS...,YTN,2021.12.28. 오전 4:10,"\n\n\n\n동영상 뉴스\n\n\n\n\n인공지능 로봇, 터널 돌아다니며 균열 정..."
3,초등 1·2학년 17%가 이용한 인공지능 활용 수학수업 지원시스템,https://news.naver.com/main/read.naver?mode=LS...,뉴스1,2021.12.28. 오전 6:00,\n\n\n\n\n교육부 '똑똑! 수학탐험대' 운영성과보고회 개최누적가입자 16.7...
4,"""인공지능株가 내년 증시 주도""…증권가가 주목한 투자는?",https://news.naver.com/main/read.naver?mode=LS...,한국경제,2021.12.28. 오후 3:36,\n\n\n\n\n이 기사는 국내 최대 해외 투자정보 플랫폼 “한경 글로벌마켓”에 ...
5,중국 인공지능(AI) 기소검사 개발…“정확도 97% 이상” 주장,https://news.naver.com/main/read.naver?mode=LS...,서울신문,2021.12.28. 오후 3:12,\n\n\n\n\n\t\n\t[서울신문 나우뉴스]중국 인공지능(AI) 기소검사 개발...
6,인공지능 앰뷸런스 살펴보는 임혜숙 장관,https://news.naver.com/main/read.naver?mode=LS...,뉴스1,2021.12.28. 오후 5:23,\n\n\n\n\n(서울=뉴스1) 신웅수 기자 = 임혜숙 과학기술정보통신부 장관이 ...
